# 1. Etração dos dados

### imports

In [157]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

### Criando conexão os bancos de dados (transacional e DW)

In [158]:
DB_HOST_ORIGEM = "localhost"
DB_NAME_ORIGEM = "crm_transacional_db"
DB_USER_ORIGEM = "crm_user"
DB_PASSWORD_ORIGEM = "crm_password"
DB_PORT_ORIGEM = "5432"

conexao_origem_str = (
    f"postgresql://{DB_USER_ORIGEM}:{DB_PASSWORD_ORIGEM}@"
    f"{DB_HOST_ORIGEM}:{DB_PORT_ORIGEM}/{DB_NAME_ORIGEM}"
)
engine_origem = create_engine(conexao_origem_str)


DB_HOST_DW = "localhost"
DB_NAME_DW = "dw_db"
DB_USER_DW = "dw_user"
DB_PASSWORD_DW = "dw_password"
DB_PORT_DW = "5433"

conexao_dw_str = (
    f"postgresql://{DB_USER_DW}:{DB_PASSWORD_DW}@"
    f"{DB_HOST_DW}:{DB_PORT_DW}/{DB_NAME_DW}"
)
engine_dw = create_engine(conexao_dw_str)

### Consumindo dados

In [159]:
query_clientes = "SELECT * FROM CLIENTE"
query_categoria_clientes = "SELECT * FROM CATEGORIA_CLIENTE"
query_categoria_produto = "SELECT * FROM CATEGORIA_PRODUTO"
query_fornecedores = "SELECT * FROM FORNECEDORES"
query_item_vendas = "SELECT * FROM ITEM_VENDAS"
query_localidade = "SELECT * FROM LOCALIDADE"
query_lojas = "SELECT * FROM LOJAS"
query_produtos = "SELECT * FROM PRODUTO"
query_produto_fornecedor = "SELECT * FROM PRODUTO_FORNECEDOR"
query_promocoes = "SELECT * FROM PROMOCOES"
query_vendas = "SELECT * FROM VENDAS"
query_vendedor = "SELECT * FROM VENDEDOR"


df_clientes = pd.read_sql(query_clientes, engine_origem)
df_categoria_clientes = pd.read_sql(query_categoria_clientes, engine_origem)
df_categoria_produto = pd.read_sql(query_categoria_produto, engine_origem)
df_fornecedores = pd.read_sql(query_fornecedores, engine_origem)
df_item_vendas = pd.read_sql(query_item_vendas, engine_origem)
df_localidade = pd.read_sql(query_localidade, engine_origem)
df_lojas = pd.read_sql(query_lojas, engine_origem)
df_produtos = pd.read_sql(query_produtos, engine_origem)
df_produto_fornecedor = pd.read_sql(query_produto_fornecedor, engine_origem)
df_promocoes = pd.read_sql(query_promocoes, engine_origem)
df_vendas = pd.read_sql(query_vendas, engine_origem)
df_vendedor = pd.read_sql(query_vendedor, engine_origem)

### Verificando nome das colunas

In [160]:
dataframes = {
        "df_clientes": df_clientes,
        "df_categoria_clientes": df_categoria_clientes,
        "df_categoria_produto": df_categoria_produto,
        "df_fornecedores": df_fornecedores,
        "df_item_vendas": df_item_vendas,
        "df_localidade": df_localidade,
        "df_lojas": df_lojas,
        "df_produtos": df_produtos,
        "df_produto_fornecedor": df_produto_fornecedor,
        "df_promocoes": df_promocoes,
        "df_vendas": df_vendas,
        "df_vendedor": df_vendedor,
}

for df_name, df in dataframes.items():
        print(f"--- DataFrame: {df_name} ---")
        print("\nColunas:")
        print(df.columns.tolist())
        print("\n" + "="*50 + "\n")

--- DataFrame: df_clientes ---

Colunas:
['id_cliente', 'nome_cliente', 'idade', 'genero', 'id_categoria_cliente', 'id_localidade']


--- DataFrame: df_categoria_clientes ---

Colunas:
['id_categoria_cliente', 'nome_categoria_cliente']


--- DataFrame: df_categoria_produto ---

Colunas:
['id_categoria_produto', 'nome_categoria_produto']


--- DataFrame: df_fornecedores ---

Colunas:
['id_fornecedor', 'nome_fornecedor', 'pais_origem']


--- DataFrame: df_item_vendas ---

Colunas:
['id_venda', 'id_produto', 'qtd_vendida', 'preco_venda', 'id_promocao_aplicada']


--- DataFrame: df_localidade ---

Colunas:
['id_localidade', 'cidade', 'estado', 'regiao']


--- DataFrame: df_lojas ---

Colunas:
['id_loja', 'nome_loja', 'gerente_loja', 'cidade', 'estado']


--- DataFrame: df_produtos ---

Colunas:
['id_produto', 'nome_produto', 'id_categoria_produto']


--- DataFrame: df_produto_fornecedor ---

Colunas:
['id_produto', 'id_fornecedor', 'custo_compra_unitario']


--- DataFrame: df_promocoes ---

# 2. Transformação dos dados

### Dimensão clientes

In [161]:
# Idades que são nulas, menores que 1 e maiores que 100 recebem a média das idades como valor. Idade passa a ser inteiro.

idade_para_media = df_clientes['idade'].copy()

idade_para_media[idade_para_media < 1] = np.nan
idade_para_media[idade_para_media > 100] = np.nan
media_idades_validas = idade_para_media.mean()
media_idades_validas = round(media_idades_validas)

condicao_idade_invalida = (df_clientes['idade'] < 1) | \
                          (df_clientes['idade'] > 100) | \
                          (df_clientes['idade'].isnull())

df_clientes.loc[condicao_idade_invalida, 'idade'] = media_idades_validas
df_clientes['idade'] = df_clientes['idade'].astype(int)

# Corrigindo valores para gênero

mapeamento_genero = {
    'F': 'Feminino',
    'M': 'Masculino',
    'Feminino': 'Feminino',
    'Masculino': 'Masculino',
    'outro': 'Outro'
}

df_clientes['genero'] = df_clientes['genero'].replace(mapeamento_genero)
df_clientes['genero'] = df_clientes['genero'].fillna('Não informado')

# Adicionando valor de categoria ao cliente

df_clientes = pd.merge(
    df_clientes,
    df_categoria_clientes,
    on='id_categoria_cliente',
    how='left'
)
df_clientes['nome_categoria_cliente'] = df_clientes['nome_categoria_cliente'].fillna('Categoria não informada')


# Criando dimensão do cliente

dim_clientes = df_clientes[[
    'id_cliente',
    'nome_cliente',
    'idade',
    'genero',
    'nome_categoria_cliente'
]].rename(columns={
    'id_cliente': 'id',
    'nome_cliente': 'nome',
    'nome_categoria_cliente': 'categoria'
})



### Dimensões de cidade, estado e região

In [164]:
# Padronização dos valores (cidade, estado e região)

df_localidade['cidade'] = df_localidade['cidade'].str.lower()
df_localidade['cidade'] = df_localidade['cidade'].str.strip()

df_localidade['regiao'] = df_localidade['regiao'].str.lower()
df_localidade['regiao'] = df_localidade['regiao'].str.strip()

df_localidade['estado'] = df_localidade['estado'].str.upper()
df_localidade['estado'] = df_localidade['estado'].str.strip()

# Adição de ID para cidade, estado e região

df_localidade['id_cidade'] = pd.factorize(df_localidade['cidade'])[0] + 1
df_localidade['id_regiao'] = pd.factorize(df_localidade['regiao'])[0] + 1
df_localidade['id_estado'] = pd.factorize(df_localidade['estado'])[0] + 1

# Criação de dimensões

dim_cidade = df_localidade[['cidade', 'id_cidade']].drop_duplicates(subset=['id_cidade'])
dim_estado = df_localidade[['estado', 'id_estado']].drop_duplicates(subset=['id_estado'])
dim_regiao = df_localidade[['regiao', 'id_regiao']].drop_duplicates(subset=['id_regiao'])


### Dimensão Produto

In [167]:
# Adição das colunas de categoria_produto ao dataframe
df_produtos = pd.merge(
    df_produtos,
    df_categoria_produto,
    on='id_categoria_produto',
    how='left'
)

# Tratamento de valores None e vazio no nome do produto
nomes_produtos = {
    '': 'Nome não informado',
    None: 'Nome não informado',
}
df_produtos['nome_produto'] = df_produtos['nome_produto'].replace(nomes_produtos)


# Tratamento do valores None e NaN na categoria do produto
df_produtos['nome_categoria_produto'] = df_produtos['nome_categoria_produto'].fillna('Sem categoria')


### Tabela fatos_vendas

In [180]:
# União de vendas com itens_venda
fato_vendas = pd.merge(
    df_vendas,
    df_item_vendas,
    on='id_venda',
    how='inner',
)


# Tratamento da coluna de data de vendas
fato_vendas['data_venda'] = fato_vendas['data_venda'].replace(['N/A', 'Data Inválida', '', None, 'None'], np.nan)
fato_vendas['data_venda'] = pd.to_datetime(fato_vendas['data_venda'], format='%Y-%m-%d', errors='coerce')


# Tratamento da coluna de quantidade de vendas
condicao_quantidade_produto = (fato_vendas['qtd_vendida'] < 1)
fato_vendas.loc[condicao_quantidade_produto, 'qtd_vendida'] = None
fato_vendas['qtd_vendida'] = fato_vendas['qtd_vendida'].astype('Int64')


# Criação das colunas de localidade do cliente
df_localidades_cliente = pd.merge(
    df_clientes,
    df_localidade,
    on='id_localidade',
    how='inner',
)

df_localidades_cliente = df_localidades_cliente.rename(columns={
    'id_cidade': 'id_cidade_cliente',
    'id_estado': 'id_estado_cliente',
    'id_regiao': 'id_regiao_cliente',
})[['id_cliente', 'id_cidade_cliente', 'id_estado_cliente', 'id_regiao_cliente']]


# Adição das colunas de id_cidade, id_estado, id_regiao a fato_vendas
fato_vendas = pd.merge(
    fato_vendas,
    df_localidades_cliente,
    on='id_cliente',
    how='left',
)

fato_vendas = fato_vendas.drop(['valor_total'], axis='columns')